<span style="color:White;font-size:28px;background-image: url(&quot;https://storage.googleapis.com/kaggle-competitions/kaggle/19990/logos/header.png?t=2020-08-17-20-10-07&quot;);" > Lyft Motion Prediction for Autonomous Vehicles</span>



### WORK IN PROGRESS 

Code Reference - https://github.com/lyft/l5kit/blob/master/examples/
                 https://github.com/lyft/l5kit

Autonomous vehicles (AVs) are expected to dramatically redefine the future of transportation. However, there are still significant engineering challenges to be solved before one can fully realize the benefits of self-driving cars. One such challenge is building models that reliably predict the movement of traffic agents around the AV, such as cars, cyclists, and pedestrians.

The ridesharing company Lyft started Level 5 to take on the self-driving challenge and build a full self-driving system (they’re hiring!). Their previous competition tasked participants with identifying 3D objects, an important step prior to detecting their movement. Now, they’re challenging you to predict the motion of these traffic agents.

In this competition, you’ll apply your data science skills to build motion prediction models for self-driving vehicles. You'll have access to the largest Prediction Dataset ever released to train and test your models. Your knowledge of machine learning will then be required to predict how cars, cyclists,and pedestrians move in the AV's environment.

Lyft’s mission is to improve people’s lives with the world’s best transportation. They believe in a future where self-driving cars make transportation safer, environment-friendly and more accessible for everyone. Their goal is to accelerate development across the industry by sharing data with researchers. As a result of your participation, you can have a hand in propelling the industry forward and helping people around the world benefit from self-driving cars sooner.

<span style='color:blue;font-size:22px' > Please upvote this kernel if you like it and leave your comments for further improvement ! </span>



In [ ]:
from IPython.display import HTML

In [ ]:
HTML('<iframe  width="850" height="450" src="https://www.youtube.com/embed/K0H43N-Hx7w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
### !pip install zarr
!pip install pymap3d==2.1.0
!pip install -U l5kit

In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import cv2
import os
#import zarr
# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18
from typing import Dict

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data


import warnings
warnings.filterwarnings("ignore")

In [ ]:
DIR_PATH = '../input/lyft-motion-prediction-autonomous-vehicles'
os.listdir(DIR_PATH)

### Understanding the dataset

In [ ]:
os.environ["L5KIT_DATA_FOLDER"] = DIR_PATH
dm = LocalDataManager(None)

# get config
cfg = load_config_data("../input/lyft-config-files/agent_motion_config.yaml")
rasterizer = build_rasterizer(cfg, dm)

#train
train_dataset = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
train_dataloader = DataLoader(AgentDataset(cfg, train_dataset, rasterizer),
                              shuffle=cfg["train_data_loader"]["shuffle"],
                              batch_size=cfg["train_data_loader"]["batch_size"],
                              num_workers=cfg["train_data_loader"]["num_workers"])

# validation
val_dataset = ChunkedDataset(dm.require(cfg["val_data_loader"]["key"])).open()
val_dataloader = DataLoader(AgentDataset(cfg, val_dataset, rasterizer),
                              shuffle=cfg["val_data_loader"]["shuffle"],
                              batch_size=cfg["val_data_loader"]["batch_size"],
                              num_workers=cfg["val_data_loader"]["num_workers"])




print(train_dataset)
print(" \n ")
print(cfg)
print(" \n ")
print(val_dataset)

# reference taken from - https://www.kaggle.com/pestipeti/pytorch-baseline-train/data

In [ ]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg: Dict):
        super().__init__()
        
        self.backbone = resnet18(pretrained=True, progress=True)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512

        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x

In [ ]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = LyftModel(cfg)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss(reduction="none")

In [ ]:
tr_it = iter(train_dataloader)

for itr in range(cfg["train_params"]["max_num_steps"]):

    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)

    model.train()
    torch.set_grad_enabled(True)
    
    # Forward pass
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)

    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

del tr_it

In [ ]:
model.eval()

In [ ]:
import gc
future_coords_offsets_pd = []
timestamps = []
agent_ids = []

with torch.no_grad():  
    
    for data in val_dataloader:

        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        targets = data["target_positions"].to(device)

        outputs = model(inputs).reshape(targets.shape)
        
        future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())
        
        del inputs ,target_availabilities , outputs , targets
        gc.collect()

In [ ]:
#submission
from l5kit.evaluation import write_pred_csv
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))